In [1]:
import os, sys, glob
ROOT_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
os.chdir(ROOT_DIR)
sys.path.append(ROOT_DIR)

In [2]:
import pandas as pd
import numpy as np
from config import settings as st
from config import path_operations as po

# defining output folder

str(sys.argv[1]), str(sys.argv[2])

output_folder = po.define_folder(os.path.join(st.DIR_DATA_PROCESSED, 'hsa/hipathia'))

config/settings.py IMPORTED!!!


In [71]:
species = 'hsa'
source = 'hipathia'

# defining output folder
output_folder = po.define_folder( os.path.join(st.DIR_DATA_PROCESSED, species, source ) )
print(output_folder)
# importing raw dataset which is imported by hipathia
df_entrez_symbol = pd.read_csv(os.path.join(st.DIR_DATA_PROCESSED, species, source, 'entrez_and_symbol.csv'))
print(df_entrez_symbol.shape)
print(df_entrez_symbol.head())

print('Checking NA values, number of NA values, ', len(df_entrez_symbol.loc[df_entrez_symbol['symbol'].isna()]))

./data/processed/hsa/hipathia/
(2987, 2)
     gene_id   symbol
0      10000     AKT3
1      10010     TANK
2  100132074    FOXO6
3  100132285  KIR2DS2
4  100132463   CLDN24
Checking NA values, number of NA values,  0


In [73]:
for i_pathway in sorted(glob.glob(os.path.join(st.DIR_DATA_PROCESSED, species, source, 'details/*gene_list.txt'))):
#     Reading selected pathways which shows the gene relation for each circuits
    df_i_pathway = pd.read_csv(i_pathway, index_col=0).fillna(value=0)
#     Replace columns names which is circuit name with pathway name
    df_i_pathway.columns = [pw[1] for pw in df_i_pathway.columns.str.split('-')]
#     Grouping all circuit as pathway representation
    df_i_pathway = df_i_pathway.groupby(df_i_pathway.columns, axis=1).max()
#     Merging entrez_and_symbol dataset with pathway information dataset
    df_entrez_symbol = pd.merge(left=df_entrez_symbol, right=df_i_pathway, left_on='gene_id', right_index=True, how='left')
    df_entrez_symbol.fillna(value=0, inplace=True)

# Updating 'symbol' values as lowercase
df_entrez_symbol['symbol'] = df_entrez_symbol['symbol'].str.lower()    
df_entrez_symbol.drop(columns=['gene_id'], inplace=True)

# EXPORTING - the prior biological knowledge layer
print('The prior biological knowledge layer EXPORTED!! - {}'.format(os.path.join(st.DIR_DATA_PROCESSED, species, source, 'bio_layer.txt')))
df_entrez_symbol.to_csv(os.path.join(st.DIR_DATA_PROCESSED, species, source, 'bio_layer.txt'), index=False)

The prior biological knowledge layer EXPORTED!! - ./data/processed/hsa/hipathia/bio_layer.txt
